In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import keras
from keras.models import Sequential
from keras import models, layers
from keras import backend as K

Using TensorFlow backend.


In [2]:
# define datasets
train_data_dir = "../data/seg_train/"
validation_data_dir = "../data/seg_test/"

# dimensions of our images.
img_width, img_height = 150, 150

nb_train_samples = 14034
nb_validation_samples = 3000
epochs = 1
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [48]:
# LeNet-5 architecture

model = Sequential()

# C1 layer: n_filters = 6, kernel = 5x5 kernel, stride = 1
model.add(layers.Conv2D(6, 5, input_shape=input_shape, padding="same", activation="tanh"))

# S2 layer: filter size = 2x2, strides = 2
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

# C3 layer: n_filters = 16, kernel = 5x5 kernel, stride = 1 
model.add(layers.Conv2D(16, 5, activation="tanh"))

# S4 layer: filter size = 2x2, strides = 2
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

# C5 layer: n_filters = 120, kernel = 5x5 kernel, stride = 1 
model.add(layers.Conv2D(120, 5, activation="tanh"))
model.add(layers.Flatten())

# FC6 layer: FC = 84
model.add(layers.Dense(84, activation="tanh"))

# FC7 layer: FC = 6
model.add(layers.Dense(6, activation="softmax"))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer="SGD", metrics=['accuracy'])

In [49]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save_weights('lenet_weights.h5')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Epoch 1/1
219/219 [==============================] - 123s 562ms/step - loss: 1.9733 - acc: 0.2185 - val_loss: 1.7900 - val_acc: 0.1671


In [52]:
test_score = model.evaluate_generator(validation_generator,
    steps = nb_validation_samples // batch_size,
    use_multiprocessing=True)

print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Test loss 1.7899, accuracy 16.75%
